In [1]:
import populator

In [2]:
data_dir = 'data'
# Second item in tuple is max batch size for requests.
exchanges = [('binance', 1000)]
tick_size = '1h'
start = '2000-01-01T00:00:00Z'

In [3]:
stables = ['TUSD/USDT', 'PAX/USDT', 'USDC/USDT', 'USDC/TUSD', 'USDS/TUSD', 'USDC/PAX', 'USDS/USDT']
populator.populate(data_dir, exchanges, stables, tick_size, start)
frames = [(pair, populator.load_data_as_frame(data_dir, 'binance', pair, tick_size, start)) for pair in stables]

Batch Size: 1000
Total Entries: 168337
Note: Entries will not exist if they predate exchange history.
Entries Processed: 162427
Entries Processed: 163427
Entries Processed: 164427
Entries Processed: 165437
Entries Processed: 166437
Entries Processed: 167437
Entries Processed: 168337
Scraping for data/binance/TUSD-USDT-1h-2000-01-01T00-00-00Z-all.csv succeeded.
Batch Size: 1000
Total Entries: 168337
Note: Entries will not exist if they predate exchange history.
Entries Processed: 165444
Entries Processed: 166444
Entries Processed: 167444
Entries Processed: 168337
Scraping for data/binance/PAX-USDT-1h-2000-01-01T00-00-00Z-all.csv succeeded.
Batch Size: 1000
Total Entries: 168337
Note: Entries will not exist if they predate exchange history.
Entries Processed: 167155
Entries Processed: 168155
Entries Processed: 168337
Scraping for data/binance/USDC-USDT-1h-2000-01-01T00-00-00Z-all.csv succeeded.
Batch Size: 1000
Total Entries: 168337
Note: Entries will not exist if they predate exchange h

In [4]:
import backtrader as bt

In [5]:
starting_cash = 10**5
class StableCoinStrategy(bt.Strategy):
    def next(self):
        if not self.position:
            # Backtrader craps out for some reason with round numbers...
            self.buy(size=starting_cash / 2 + 1)
        elif self.data.close[0] > 1.0 and self.position.size >= 1000:
            self.sell(size=1000)
        elif self.data.close[0] < 1.0 and self.broker.getcash() * self.data.close[0] >= 1000:
            self.buy(size=1000)

In [6]:
def run_strategy(name, df):
    print('Running strategy for {}.'.format(name))
    cerebro = bt.Cerebro()
    cerebro.addstrategy(StableCoinStrategy)
    data = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(data)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.00, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

    cerebro.broker.set_coc(True)
    cerebro.broker.setcash(starting_cash)
    cerebro.broker.setcommission(commission=0.001)
    sim = cerebro.run()
    
    print('Final Value: {}'.format(cerebro.broker.getvalue()))
    print('PNL %: {}'.format(100.0 * (cerebro.broker.getvalue() / starting_cash) - 100.0))
    print('Sharpe Ratio: {}'.format(sim[0].analyzers.sharpe.get_analysis()['sharperatio']))
    print('Max Drawdown: {}'.format(sim[0].analyzers.drawdown.get_analysis().max.drawdown))
    print()

In [7]:
for (name, df) in frames:
    run_strategy(name, df)

Running strategy for TUSD/USDT.
Final Value: 100605.99699599993
PNL %: 0.6059969959999307
Sharpe Ratio: 1.0458326871925159
Max Drawdown: 1.8092908826678178

Running strategy for PAX/USDT.
Final Value: 100269.76368909994
PNL %: 0.2697636890999462
Sharpe Ratio: 1.29022679956778
Max Drawdown: 1.8282380397804032

Running strategy for USDC/USDT.
Final Value: 100437.39268829995
PNL %: 0.43739268829993705
Sharpe Ratio: 0.919302002269944
Max Drawdown: 1.8921819655301215

Running strategy for USDC/TUSD.
Final Value: 99534.10270000022
PNL %: -0.46589729999976726
Sharpe Ratio: None
Max Drawdown: 1.0698793000000588

Running strategy for USDS/TUSD.
Final Value: 99692.18170080044
PNL %: -0.3078182991995533
Sharpe Ratio: None
Max Drawdown: 0.4266194991995435

Running strategy for USDC/PAX.
Final Value: 99716.19249800013
PNL %: -0.2838075019998598
Sharpe Ratio: None
Max Drawdown: 0.35653567032578254

Running strategy for USDS/USDT.
Final Value: 99984.37089260003
PNL %: -0.01562910739997392
Sharpe Rati